In [1]:
import sympy as sym
from IPython.display import display, Math, Latex
import pandas as pd
import numpy as np
def func_str(function):
    return str(function)
    function_str = str(function)
    function_str = function_str.replace("**", "^")
    return function_str
class Iterative:
    def __init__(self, function, *args):
        self.variables = []
        self.function = function
        self.data_dict = {"iterate":[],"G_val":[]}
        for variable in args:
            self.variables.append(variable)
            self.data_dict["der_" + str(variable)] = []
            self.data_dict[str(variable)] = []
        self.partial_deravatives()

    def partial_deravatives(self):
         self.deravatives = []
         i = 0
         for variable in self.variables:
             self.deravatives.append(sym.diff(self.function, variable))
             display(Math(r"\frac{\partial G(\mathbf{Y_k})}{\partial y_%d}" % (i+1) + " = " + str(self.deravatives[i])))
             i += 1
    
    def calc_limit_state(self, numerical_vars):
        temp_dict = {}
        to_show = "G("
        for index,variable in enumerate(self.variables):
            temp_dict[variable] =numerical_vars[index]
            if index != (len(self.variables)-1):
                to_show += str(variable) + " = " + str(numerical_vars[index]) + ", "
            else:
                to_show += str(variable) + " = " + str(numerical_vars[index]) + ")"
        display(Math(to_show)) 
        return self.function.subs(temp_dict)


    def calc_gradient(self, numerical_vars):
        gradient = []
        temp_dict = {}
        for index, variable in enumerate(self.variables):
            temp_dict[variable] = numerical_vars[index]
        
        for deravative in self.deravatives:
            gradient.append(deravative.subs(temp_dict))
        
        return gradient


    def make_dataframe(self):
        return pd.DataFrame(data=self.data_dict)

    def iterate(self):
        i = 0
        self.ys = []
        while(True):
            if(i == 0):
                for j in range(len(self.variables)):
                    self.ys.append(0)
            self.data_dict["iterate"].append(i)
            i += 1
            gradient = np.array(self.calc_gradient(self.ys))
            limit_state = self.calc_limit_state(self.ys)
            ys = np.array(self.ys).reshape(-1,1)
            gradient_measurement = sum(gradient**2)
            self.ys = (1/gradient_measurement)*(gradient.dot(ys)[0] - limit_state) * gradient
            self.data_dict["G_val"].append(limit_state)
            for index in range(len(self.variables)):
                self.data_dict["der_" + str(self.variables[index])].append(gradient[index])
                self.data_dict[str(self.variables[index])].append(ys[index][0])

            if abs(limit_state) < 0.001:
                break
        
    
    def get_beta(self):
        self.beta = (sum(self.ys**2))**0.5
        display(Math(r"\beta = " + str(self.beta)))
        return self.beta
        
    

        


y_1, y_2 = sym.symbols('y_1 y_2')

G = 10 * y_1 + 15 - (2 * (3**0.5)) * y_2 - 3

iterative = Iterative(G, y_1, y_2)
iterative.iterate()
iterative.get_beta()
iterative.make_dataframe().round(3)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

,iterate,G_val,der_y_1,y_1,der_y_2,y_2
0,0,12,10,0,-3.46410161513775,0
1,1,8.88178419700125e-16,10,-1.07142857142857,-3.46410161513775,0.371153744479045
